In [78]:
import nltk; nltk.download('stopwards')

[nltk_data] Error loading stopwards: Package 'stopwards' not found in
[nltk_data]     index


False

In [79]:
import re
import numpy as np
import pandas as pd

#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy for lemmatization
import spacy

#plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline


#optional
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [80]:
#NLTK Stopwords

from nltk.corpus import stopwords
stop_wards = stopwords.words('english')
print(stop_wards)
stop_wards.extend(['from', 'use'])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [81]:
raw_corpus = []
for i in range(1,10):
    filename = str(i) + '.txt'
    f = open(filename,'r')#file opened
    temp = f.read()
    #raw_corpus[i] = temp
    raw_corpus.append(temp)
#print(raw_corpus)
#print(np.shape(raw_corpus))


In [82]:
# Create a set of frequent words
#stoplist = set('for a of the and to in'.split(' '))
#texts = raw_corpus.lower().split()
texts = [[word for word in document.lower().split() if word not in stop_wards]
         for document in raw_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)

for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
print(processed_corpus)

[['academy', 'management', 'journal', '2013,', 'vol.', '56,', 'no.', '2,', 'emotion', 'helping', 'managers:', 'emergent', 'understanding', 'discrepant', 'role', 'expectations', 'outcomes', 'ginka', 'toegel', 'imd,', 'lausanne', 'martin', 'kilduff', 'university', 'college', 'london', 'n.', 'anand', 'imd,', 'lausanne', 'managers', 'help', 'employees', 'negative', 'emotions,', 'employees', 'analyzed', 'interview', 'network', 'data', 'head', 'office', 'recruiting', 'agency.', 'found', 'managers', 'active', 'provision', 'emotion', 'help', 'thought', 'help', 'managerial', 'whereas', 'employees', 'defined', 'emotional', 'support', 'managerial', 'in-role', 'behavior.', 'parties', 'accepted', 'necessity', 'controlling', 'negative', 'emotions', 'good', 'organization.', 'helped', 'tended', 'perceive', 'helpers', 'doubly', 'powerful', 'figures,', 'invested', 'formal', 'authority', 'parental', 'authority,', 'whereas', 'helpers', 'saw', 'situational', 'contingencies', 'favors', 'subordinates', 'need

In [83]:
bigram = gensim.models.Phrases(processed_corpus, min_count=5, threshold=100) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)
print(bigram_mod[processed_corpus[0]])



['academy_management', 'journal', '2013,', 'vol.', '56,', 'no._2,', 'emotion_helping', 'managers:', 'emergent', 'understanding', 'discrepant', 'role', 'expectations', 'outcomes', 'ginka', 'toegel', 'imd,', 'lausanne', 'martin', 'kilduff', 'university', 'college', 'london', 'n.', 'anand', 'imd,', 'lausanne', 'managers', 'help', 'employees', 'negative', 'emotions,', 'employees', 'analyzed', 'interview', 'network', 'data', 'head', 'office', 'recruiting', 'agency.', 'found', 'managers', 'active', 'provision_emotion', 'help', 'thought', 'help', 'managerial', 'whereas', 'employees', 'defined', 'emotional', 'support', 'managerial', 'in-role', 'behavior.', 'parties', 'accepted', 'necessity', 'controlling', 'negative_emotions', 'good', 'organization.', 'helped', 'tended', 'perceive', 'helpers', 'doubly', 'powerful', 'figures,', 'invested', 'formal', 'authority', 'parental', 'authority,', 'whereas', 'helpers', 'saw', 'situational', 'contingencies', 'favors', 'subordinates', 'need.', 'results', '

In [84]:
trigram = gensim.models.Phrases(bigram_mod[processed_corpus], threshold=100) # higher threshold fewer phrases.
trigram_mod = gensim.models.phrases.Phraser(trigram)
print(trigram_mod[bigram_mod[processed_corpus[0]]])

['academy_management_journal', '2013,', 'vol.', '56,', 'no._2,', 'emotion_helping', 'managers:', 'emergent', 'understanding', 'discrepant', 'role', 'expectations', 'outcomes', 'ginka', 'toegel', 'imd,', 'lausanne', 'martin', 'kilduff', 'university', 'college', 'london', 'n.', 'anand', 'imd,', 'lausanne', 'managers', 'help', 'employees', 'negative', 'emotions,', 'employees', 'analyzed', 'interview', 'network', 'data', 'head', 'office', 'recruiting', 'agency.', 'found', 'managers', 'active', 'provision_emotion_help', 'thought', 'help', 'managerial', 'whereas', 'employees', 'defined', 'emotional', 'support', 'managerial', 'in-role', 'behavior.', 'parties', 'accepted', 'necessity', 'controlling', 'negative_emotions', 'good', 'organization.', 'helped', 'tended', 'perceive', 'helpers', 'doubly', 'powerful', 'figures,', 'invested', 'formal', 'authority', 'parental', 'authority,', 'whereas', 'helpers', 'saw', 'situational', 'contingencies', 'favors', 'subordinates', 'need.', 'results', 'point'

In [85]:
def make_bigrams(text2):
    return [bigram_mod[doc] for doc in text2]

def make_trigrams(text2):
    return [trigram_mod[bigram_mod[doc]] for doc in text2] 
def lemmatization(text2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in text2:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [86]:
corpus_bi = make_bigrams(processed_corpus)
#print(corpus_bi)
corpus_tri = make_bigrams(corpus_bi)

print(np.shape(corpus_tri))
print(corpus_tri)

(9,)
[['academy_management', 'journal', '2013,', 'vol.', '56,', 'no._2,', 'emotion_helping', 'managers:', 'emergent', 'understanding', 'discrepant', 'role', 'expectations', 'outcomes', 'ginka', 'toegel', 'imd,', 'lausanne', 'martin', 'kilduff', 'university', 'college', 'london', 'n.', 'anand', 'imd,', 'lausanne', 'managers', 'help', 'employees', 'negative', 'emotions,', 'employees', 'analyzed', 'interview', 'network', 'data', 'head', 'office', 'recruiting', 'agency.', 'found', 'managers', 'active', 'provision_emotion', 'help', 'thought', 'help', 'managerial', 'whereas', 'employees', 'defined', 'emotional', 'support', 'managerial', 'in-role', 'behavior.', 'parties', 'accepted', 'necessity', 'controlling', 'negative_emotions', 'good', 'organization.', 'helped', 'tended', 'perceive', 'helpers', 'doubly', 'powerful', 'figures,', 'invested', 'formal', 'authority', 'parental', 'authority,', 'whereas', 'helpers', 'saw', 'situational', 'contingencies', 'favors', 'subordinates', 'need.', 'resul

In [87]:
nlp = spacy.load('en', disable=['parser', 'ner'])
corpus = lemmatization(corpus_tri, allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV'])

print(np.shape(corpus))
print(corpus)

(9,)
[['academy_management', 'journal', 'no._2', 'emotion_help', 'manager', 'emergent', 'understand', 'discrepant', 'role', 'expectation', 'outcome', 'ginka', 'toegel', 'imd', 'lausanne', 'martin', 'kilduff', 'university', 'college', 'london', 'n.', 'anand', 'imd', 'lausanne', 'manager', 'help', 'employee', 'negative', 'emotion', 'employee', 'analyze', 'interview', 'network', 'datum', 'head', 'office', 'recruiting', 'agency', 'find', 'manager', 'active', 'provision_emotion', 'help', 'think', 'help', 'managerial', 'employee', 'define', 'emotional', 'support', 'managerial', 'role', 'behavior', 'party', 'accept', 'necessity', 'control', 'negative_emotion', 'good', 'organization', 'help', 'tend', 'perceive', 'helper', 'doubly', 'powerful', 'figure', 'invest', 'formal', 'authority', 'parental', 'authority', 'helper', 'see', 'situational', 'contingency', 'favor', 'subordinate', 'need', 'result', 'point', 'emergent', 'understand', 'discrepant', 'employee', 'treat', 'caring', 'part', 'managers

In [88]:
#from gensim import corpora
print(type(corpus))
dictionary = corpora.Dictionary(corpus[:])
print(list(dictionary))

<class 'list'>
[1816, 3455, 4894, 5304, 5130, 4933, 388, 3528, 2029, 2358, 3, 1097, 3111, 4302, 1568, 2710, 2387, 1304, 1189, 3014, 5224, 4267, 3696, 4324, 1641, 4207, 4255, 120, 282, 2982, 2586, 5127, 1401, 1242, 458, 2688, 1345, 5221, 1110, 5042, 1726, 3082, 2870, 3040, 1141, 1100, 2144, 3803, 5110, 141, 3479, 4588, 931, 2612, 1094, 1497, 3874, 5104, 5239, 3418, 4203, 3060, 5212, 3954, 745, 4186, 702, 4940, 950, 4184, 4738, 2303, 905, 4853, 1088, 1409, 3975, 957, 3661, 1191, 1285, 1013, 5339, 1217, 2811, 207, 1495, 2884, 2537, 4956, 723, 2951, 725, 2736, 459, 3075, 2910, 4755, 1778, 777, 1005, 3561, 1794, 2098, 1312, 1275, 893, 2422, 3213, 3458, 3705, 4851, 4144, 3302, 4841, 4312, 58, 786, 4198, 3571, 4775, 558, 3191, 1323, 1029, 2734, 191, 3183, 1863, 3801, 5016, 5146, 1783, 4974, 2115, 1584, 3002, 4552, 1646, 2618, 2017, 4137, 5099, 1059, 4952, 1468, 3875, 3848, 5174, 4411, 2513, 1281, 4968, 129, 4041, 2651, 230, 4529, 2368, 813, 98, 3633, 1327, 3966, 1691, 3978, 479, 4356, 2180, 2

In [89]:
print(dictionary.token2id)

{'author': 1816, 'combined': 3455, 'yangtze': 4894, 'schmeichel': 5304, 'refer-': 5130, 'am': 4933, 'decision': 388, 'eliminate': 3528, 'eager': 2029, 'official': 2358, '.85': 3, 'organiza-': 1097, 'novel': 3111, 'driven': 4302, 'tern': 1568, 'accumulation': 2710, 'perfor-_mance': 2387, 'recognition': 1304, 'port': 1189, 'innovation_rat': 3014, 'computed': 5224, 'consult': 4267, 'palmer': 3696, 'fewer': 4324, 'uhlmann': 1641, 'agence': 4207, 'collaborate': 4255, 'assure': 120, 'college': 282, 'history': 2982, 'tangible': 2586, 'ramarajan': 5127, 'second': 1401, 'prosocial': 1242, 'dou-': 458, 'zacharatos': 2688, 'request': 1345, 'compre-': 5221, 'outcomes': 1110, 'identifica-': 5042, 'workday': 1726, 'meth-': 3082, 'dj': 2870, 'judge': 3040, 'pat-': 1141, 'organizational': 1100, 'hillsdale': 2144, 'tunity': 3803, 'organ._behav': 5110, 'bacharach': 141, 'conviction': 3479, 'zald': 4588, 'maanen': 931, 'tonidandel': 2612, 'organ': 1094, 'staw': 1497, 'bound': 3874, 'norenzayan': 5104, 'e

In [90]:
eg_doc = " why why why and how managers managers engage in the process"
eg_vec = dictionary.doc2bow(eg_doc.lower().split())
eg_vec

[(534, 1), (1229, 1), (2149, 1), (3832, 3), (4369, 1)]

In [91]:
bow_corpus = [dictionary.doc2bow(text) for text in corpus]
print(bow_corpus)

[[(0, 5), (1, 2), (2, 1), (3, 1), (4, 3), (5, 26), (6, 8), (7, 2), (8, 5), (9, 3), (10, 3), (11, 4), (12, 20), (13, 2), (14, 1), (15, 3), (16, 1), (17, 6), (18, 3), (19, 3), (20, 1), (21, 5), (22, 1), (23, 2), (24, 6), (25, 2), (26, 8), (27, 4), (28, 1), (29, 5), (30, 2), (31, 2), (32, 1), (33, 2), (34, 1), (35, 2), (36, 1), (37, 4), (38, 1), (39, 1), (40, 6), (41, 1), (42, 1), (43, 1), (44, 4), (45, 4), (46, 2), (47, 6), (48, 3), (49, 5), (50, 1), (51, 3), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 2), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 2), (64, 24), (65, 1), (66, 3), (67, 1), (68, 1), (69, 2), (70, 6), (71, 2), (72, 1), (73, 1), (74, 2), (75, 3), (76, 4), (77, 1), (78, 2), (79, 12), (80, 5), (81, 1), (82, 16), (83, 1), (84, 3), (85, 3), (86, 2), (87, 2), (88, 4), (89, 1), (90, 2), (91, 3), (92, 1), (93, 1), (94, 2), (95, 3), (96, 1), (97, 10), (98, 2), (99, 1), (100, 11), (101, 3), (102, 3), (103, 1), (104, 1), (105, 2), (106, 1), (107, 1), (108, 6), (109, 1), (

In [92]:
#from gensim import models

# tfidf = models.TfidfModel(bow_corpus)

# tfidf[dictionary.doc2bow("system minors".lower().split())]

In [93]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

#coherance with
#5
# Perplexity:  -7.372994219640406

# Coherence Score:  0.29504758739506826

#10 topic - 0.326907679400502

#20 topic - Perplexity:  -7.276310852105469

            #Coherence Score:  0.38374245522686934
#15
                
# Perplexity:  -7.343511965710279

# Coherence Score:  0.33103216574231553


In [94]:
print(lda_model.print_topics())
doc_lda = lda_model[bow_corpus]

[(0, '0.015*"high" + 0.014*"low" + 0.012*"order" + 0.012*"organizational" + 0.011*"effect" + 0.011*"industry" + 0.010*"firm" + 0.010*"organization" + 0.009*"climate" + 0.009*"corporate_illegality"'), (1, '0.023*"bootlegging" + 0.017*"individual" + 0.015*"innovation" + 0.013*"r&d" + 0.012*"work" + 0.010*"may" + 0.010*"organization" + 0.010*"effort" + 0.009*"time" + 0.009*"unit"'), (2, '0.025*"firm" + 0.015*"ceo" + 0.011*"work" + 0.010*"social" + 0.010*"corporate" + 0.008*"performance" + 0.008*"decision" + 0.007*"effect" + 0.007*"activist" + 0.006*"fwc"'), (3, '0.001*"identity" + 0.001*"high" + 0.001*"j." + 0.001*"organization" + 0.001*"model" + 0.001*"firm" + 0.001*"may" + 0.001*"effect" + 0.001*"work" + 0.001*"manager"'), (4, '0.002*"incumbent" + 0.002*"entrant" + 0.002*"price" + 0.002*"firm" + 0.001*"manager" + 0.001*"response" + 0.001*"competitive" + 0.001*"effect" + 0.001*"entry" + 0.001*"may"'), (5, '0.030*"manager" + 0.023*"help" + 0.015*"emotion" + 0.014*"employee" + 0.010*"subor

In [95]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.276310852105469

Coherence Score:  0.38374245522686934


In [96]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      20.983877        1       1 -0.099069 -0.068084
17     13.296896        1       2 -0.162101  0.034446
14     12.416799        1       3 -0.141225  0.085939
5      11.573930        1       4 -0.139017  0.000455
1      11.522303        1       5 -0.139664 -0.081746
0      11.112713        1       6 -0.142062  0.101906
18     10.406942        1       7 -0.129945 -0.204231
7       8.683849        1       8 -0.144426  0.101421
13      0.000240        1       9  0.076637  0.005771
4       0.000236        1      10  0.071083  0.019748
9       0.000235        1      11  0.086757  0.002701
10      0.000228        1      12  0.090810  0.000978
11      0.000225        1      13  0.096061  0.002225
15      0.000225        1      14  0.098364  0.004424
3       0.000218        1      15  0.106083 -0.001787
8       0.000218        1      16  0.104584 -0.002654
19      0.000218        1      17  0.101643  0.002649
6       0.000218        1      18  0.095704 -0.005463
12      0.000218        1      19  0.084881  0.003931
16      0.000218        1      20  0.084903 -0.002628, topic_info=     Category        Freq               Term       Total  loglift  logprob
term                                                                      
2653  Default  559.000000               firm  559.000000  30.0000  30.0000
1636  Default  415.000000                ceo  415.000000  29.0000  29.0000
473   Default  344.000000           identity  344.000000  28.0000  28.0000
5336  Default  436.000000            manager  436.000000  27.0000  27.0000
3855  Default  528.000000               work  528.000000  26.0000  26.0000
1870  Default  201.000000           humility  201.000000  25.0000  25.0000
5113  Default  244.000000                age  244.000000  24.0000  24.0000
2782  Default  266.000000               help  266.000000  23.0000  23.0000
1315  Default  247.000000           practice  247.000000  22.0000  22.0000
4189  Default  374.000000        performance  374.000000  21.0000  21.0000
4341  Default  395.000000               high  395.000000  20.0000  20.0000
3417  Default  351.000000              model  351.000000  19.0000  19.0000
2778  Default  185.000000        bootlegging  185.000000  18.0000  18.0000
2573  Default  265.000000           employee  265.000000  17.0000  17.0000
3521  Default  386.000000             effect  386.000000  16.0000  16.0000
45    Default  357.000000     organizational  357.000000  15.0000  15.0000
3054  Default  384.000000           research  384.000000  14.0000  14.0000
3547  Default  372.000000       organization  372.000000  13.0000  13.0000
3818  Default  329.000000              study  329.000000  12.0000  12.0000
5261  Default  391.000000                may  391.000000  11.0000  11.0000
4672  Default  338.000000                 j.  338.000000  10.0000  10.0000
2921  Default  295.000000             social  295.000000   9.0000   9.0000
4489  Default  208.000000          corporate  208.000000   8.0000   8.0000
2715  Default  273.000000                low  273.000000   7.0000   7.0000
4191  Default  163.000000             mature  163.000000   6.0000   6.0000
3153  Default  142.000000  stereotype_threat  142.000000   5.0000   5.0000
2369  Default  228.000000         individual  228.000000   4.0000   4.0000
340   Default  135.000000      configuration  135.000000   3.0000   3.0000
5317  Default  125.000000            emotion  125.000000   2.0000   2.0000
6     Default  161.000000           decision  161.000000   1.0000   1.0000
...       ...         ...                ...         ...      ...      ...
4341  Topic20    0.000165               high  395.368805  -1.6517  -6.8271
1636  Topic20    0.000154                ceo  415.129181  -1.7718  -6.8985
3054  Topic20    0.000145           research  384.721008  -1.7514  -6.9542
3547  Topic20    0.000136       organization  372.456543  -1.7847  -7

In [ ]:
path = /home/rajendra/ml_programs/